In [3]:
print('hello')

hello


통계 분석

1. 품목별(Item ID)
* 평균 이동 횟수
* 최대/최소 이동량

2. 창고별(Warehouse ID)
* 평균 이동 횟수
* 최대/최소 이동량

시간별 트렌드 분석
* 일/주/월별 전체 이동량 및 이동 횟수 변화 시각화
 - 주기성이나 갑작스러운 변동 감지

* 일/주/월별 Top5 이동 품목

* 일/주/월별 Top5 이동 창고

* 입/출고 시간대 분석
 - 입고 및 출고가 특정 시간대에 집중하는지 확인
 - Movement Date

재고 회전율 분석
* 특정 기간(예: 2025년 5월)동안 각 품목이 입고/출고된 횟수
* 회전률이 낮은 품목

추가 인사이트
* Movement Type에 따라 처리 시간을 확인할 수 있는 작업 소요 필드 필요.
* 작업 로그별 오류 횟수 필드가 있다면 오류 분석에 유용할 것.


In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
data_location = './datasets/warehouse-inventory-movement-log.csv'
movement_log_df = pd.read_csv(data_location)
movement_log_df.head()

,movement_id,timestamp,item_id,item_name,sku,batch_number,quantity,uom,movement_type,source_location_id,source_location_name,destination_location_id,destination_location_name,operator_id,operator_name,reference_document,movement_reason,comments
0,MV000001,2024-03-04T08:19:00,ITM-1012,Plastic Storage Bin,SKU-PLB-002,BN-202403A,32,pieces,inbound,NaN,NaN,BIN-17C,"Aisle 5, Bin 17C",OP-003,Dana Patel,PO-20240302-11,replenishment,Morning delivery from supplier
1,MV000002,2024-03-04T09:42:15,ITM-1045,Aluminum Sheet Roll,SKU-ALM-019,ALM-2402-B,-5,rolls,outbound,RACK-2B,Rack 2B,NaN,NaN,OP-005,Luke Chang,SO-20240304-04,picking,Dispatched for order 1124
2,MV000003,2024-03-04T10:11:00,ITM-1008,Corrugated Box Large,SKU-CBX-003,NaN,100,pieces,inbound,NaN,NaN,BIN-9A,"Aisle 2, Bin 9A",OP-001,Samantha Lee,PO-20240302-06,replenishment,NaN
3,MV000004,2024-03-04T10:37:36,ITM-1030,Packing Tape Clear,SKU-PTC-015,PTC-20240301,-24,boxes,outbound,BIN-3E,"Aisle 1, Bin 3E",NaN,NaN,OP-007,Jayden Brooks,SO-20240304-07,picking,For express shipment
4,MV000005,2024-03-04T11:02:20,ITM-1024,Blue Pallet,SKU-PAL-001,NaN,12,pallets,inbound,NaN,NaN,DOCK-01,Receiving Dock 01,OP-009,Erika Schmidt,PO-20240301-27,replenishment,NaN


# 변수 정리


## 1. 분석에 필요한 데이터 남기기

In [52]:
# df = movement_log_df.iloc[:, :-4].copy()

df = movement_log_df.copy()
drop_vars = ['item_name', 'uom', 'destination_location_name',
             'operator_name', 'reference_document', 'comments']
df.drop(columns=drop_vars, axis=1, inplace=True)
df.head()

,movement_id,timestamp,item_id,sku,batch_number,quantity,movement_type,source_location_id,source_location_name,destination_location_id,operator_id,movement_reason
0,MV000001,2024-03-04T08:19:00,ITM-1012,SKU-PLB-002,BN-202403A,32,inbound,NaN,NaN,BIN-17C,OP-003,replenishment
1,MV000002,2024-03-04T09:42:15,ITM-1045,SKU-ALM-019,ALM-2402-B,-5,outbound,RACK-2B,Rack 2B,NaN,OP-005,picking
2,MV000003,2024-03-04T10:11:00,ITM-1008,SKU-CBX-003,NaN,100,inbound,NaN,NaN,BIN-9A,OP-001,replenishment
3,MV000004,2024-03-04T10:37:36,ITM-1030,SKU-PTC-015,PTC-20240301,-24,outbound,BIN-3E,"Aisle 1, Bin 3E",NaN,OP-007,picking
4,MV000005,2024-03-04T11:02:20,ITM-1024,SKU-PAL-001,NaN,12,inbound,NaN,NaN,DOCK-01,OP-009,replenishment


## 날짜에서 date, time 추출 및 시간 데이터 변환

In [ ]:
df[['date', 'time']] = df['timestamp'].str.split('T', expand=True)
for i in ['timestamp', 'date']:
    df[i] = pd.to_datetime(df[i], errors='coerce')

df['time'] = pd.to_datetime(df['time'], errors='coerce', format='%H:%M:%S').dt.time
# dt.time으로 가짜 날짜 1900-01-01 제거하고 시간 정보만 추출
df.head()

AttributeError: Can only use .str accessor with string values!

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   movement_id              200 non-null    object        
 1   timestamp                200 non-null    datetime64[ns]
 2   item_id                  200 non-null    object        
 3   sku                      200 non-null    object        
 4   batch_number             107 non-null    object        
 5   quantity                 200 non-null    int64         
 6   movement_type            200 non-null    object        
 7   source_location_id       112 non-null    object        
 8   source_location_name     112 non-null    object        
 9   destination_location_id  128 non-null    object        
 10  operator_id              200 non-null    object        
 11  movement_reason          200 non-null    object        
 12  date                     200 non-nul

In [39]:
df['movement_reason'].unique()

array(['replenishment', 'picking', 'cycle count', 'customer return',
       'shipping', 'damage', 'reallocation'], dtype=object)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   movement_id              200 non-null    object        
 1   timestamp                200 non-null    datetime64[ns]
 2   item_id                  200 non-null    object        
 3   sku                      200 non-null    object        
 4   batch_number             107 non-null    object        
 5   quantity                 200 non-null    int64         
 6   movement_type            200 non-null    object        
 7   source_location_id       112 non-null    object        
 8   source_location_name     112 non-null    object        
 9   destination_location_id  128 non-null    object        
 10  operator_id              200 non-null    object        
 11  movement_reason          200 non-null    object        
 12  date                     200 non-nul

In [27]:
df[df['movement_type'] == 'inbound']['quantity'].describe()

count     67.000000
mean      47.522388
std       58.199593
min        1.000000
25%       12.000000
50%       30.000000
75%       60.000000
max      300.000000
Name: quantity, dtype: float64

In [28]:
df[df['movement_type'] == 'outbound']['quantity'].describe()

count     60.00000
mean     -35.45000
std       46.82004
min     -200.00000
25%      -40.00000
50%      -15.00000
75%       -8.00000
max       -1.00000
Name: quantity, dtype: float64

In [14]:
df[['date', 'time']] = df['timestamp'].astype(str).str.split('T', expand=True)

In [9]:
df['timestamp'] = pd.to_datetime(df['timestamp'])